<a href="https://colab.research.google.com/github/Praneeth-18/No-Code-and-Low-Code-tools-for-GenAI-Usecases/blob/main/a.%20LLaMA%20Factory%20Experiments/LLama_Factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Supervised Fine-Tuning (LoRA/QLoRA)

In [2]:
# LLaMA Factory Colab Setup Script

# Install dependencies
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax

# Clone LLaMA Factory repository
%cd /content/
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory

# Install LLaMA Factory
!pip install -e .[torch,bitsandbytes,liger-kernel]

# Check GPU environment
import torch
assert torch.cuda.is_available(), "GPU not available. Please ensure you're using a GPU runtime in Colab."

print("GPU available:", torch.cuda.get_device_name(0))

# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

print("Setup complete. You're now ready to run the training scripts.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

Setup complete. You're now ready to run the training scripts.


Fine-Tuning with LoRA:

Configure the training parameters for supervised fine-tuning using LoRA adapters. This involves setting up your model path, datasets, and training specifics like learning rate and batch size.

In [2]:
import json

args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit",
  dataset="identity,alpaca_en_demo",
  template="llama3",
  finetuning_type="lora",
  lora_target="all",
  output_dir="llama3_lora",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_samples=500,
  max_grad_norm=1.0,
  quantization_bit=4,
  loraplus_lr_ratio=16.0,
  fp16=True,
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)
!llamafactory-cli train train_llama3.json

2024-09-30 00:42:21.731693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 00:42:21.751429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 00:42:21.757337: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 00:42:21.771687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 00:42:22.940519: W tensorflow/compiler/tf2

# 2. DPO Training
Dynamic Programming Optimization (DPO) Training:
Set up a configuration file for DPO training by specifying the model, dataset, and output directory.

In [5]:
# LLaMA Factory Colab Setup Verification

import os
import sys

# Change to the LLaMA Factory directory
%cd /content/LLaMA-Factory

# Print current directory
print("Current directory:", os.getcwd())

# List contents of current directory
print("\nContents of current directory:")
!ls -l

# Check if src directory exists
if os.path.exists('src'):
    print("\nsrc directory exists")
    print("Contents of src directory:")
    !ls -l src
else:
    print("\nsrc directory does not exist")

# Print Python path
print("\nPython path:")
print(sys.path)

# Try to import llmtuner
try:
    import llmtuner
    print("\nSuccessfully imported llmtuner")
except ImportError as e:
    print(f"\nFailed to import llmtuner: {e}")

# Check installed packages
print("\nInstalled packages:")
!pip list

# Print LLaMA Factory version (if available)
try:
    import llmtuner
    print(f"\nLLaMA Factory version: {llmtuner.__version__}")
except:
    print("\nUnable to determine LLaMA Factory version")

/content/LLaMA-Factory
Current directory: /content/LLaMA-Factory

Contents of current directory:
total 176
drwxr-xr-x  2 root root  4096 Sep 30 19:08 assets
-rw-r--r--  1 root root  1378 Sep 30 19:08 CITATION.cff
drwxr-xr-x  6 root root  4096 Sep 30 19:08 data
drwxr-xr-x  5 root root  4096 Sep 30 19:08 docker
-rw-r--r--  1 root root   645 Sep 30 19:23 dpo_config.json
drwxr-xr-x  5 root root  4096 Sep 30 19:08 evaluation
drwxr-xr-x 10 root root  4096 Sep 30 19:08 examples
-rw-r--r--  1 root root 11324 Sep 30 19:08 LICENSE
-rw-r--r--  1 root root   251 Sep 30 19:08 Makefile
-rw-r--r--  1 root root    33 Sep 30 19:08 MANIFEST.in
-rw-r--r--  1 root root   645 Sep 30 19:08 pyproject.toml
-rw-r--r--  1 root root 51040 Sep 30 19:08 README.md
-rw-r--r--  1 root root 50807 Sep 30 19:08 README_zh.md
-rw-r--r--  1 root root   292 Sep 30 19:08 requirements.txt
drwxr-xr-x  2 root root  4096 Sep 30 19:08 scripts
-rw-r--r--  1 root root  3745 Sep 30 19:08 setup.py
drwxr-xr-x  4 root root  4096 Sep 30

In [6]:
# LLaMA Factory Colab Reinstallation

# Remove existing installation
%cd /content
!rm -rf LLaMA-Factory

# Clone the repository again
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory

# Install dependencies
!pip install -e .[torch,bitsandbytes,liger-kernel]

# Verify installation
try:
    import llmtuner
    print(f"LLaMA Factory installed successfully. Version: {llmtuner.__version__}")
except ImportError:
    print("Failed to import llmtuner. Installation might have issues.")

# List contents to verify structure
print("\nContents of LLaMA-Factory directory:")
!ls -l

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 17703, done.
remote: Counting objects: 100% (8142/8142), done.
remote: Compressing objects: 100% (695/695), done.
remote: Total 17703 (delta 7659), reused 7541 (delta 7447), pack-reused 9561 (from 1)
Receiving objects: 100% (17703/17703), 226.66 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (13041/13041), done.
/content/LLaMA-Factory
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.1.dev0-0.editable-py3-none-any.whl size=22462 sha256=64f25d884abb61d03ffc3ca9bfa0aad640039727dca6e4d9ac11fdb0131bb852
  Stored in directory: /tmp/pip-ephem-wheel-cache-jhhl2zw2/wheels/de/aa/c5/27b5682c5592b7c0eecc3e208f17

In [7]:
# LLaMA Factory Import Verification

import sys
import os

# Add the LLaMA Factory directory to the Python path
llama_factory_path = '/content/LLaMA-Factory'
if llama_factory_path not in sys.path:
    sys.path.append(llama_factory_path)

# Add the src directory to the Python path
src_path = os.path.join(llama_factory_path, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Print the updated Python path
print("Updated Python path:")
print(sys.path)

# Try to import LLaMA Factory
try:
    import llamafactory
    print(f"Successfully imported LLaMA Factory. Version: {llamafactory.__version__}")
except ImportError as e:
    print(f"Failed to import LLaMA Factory: {e}")
    print("Trying alternative import...")
    try:
        from src import llmtuner
        print("Successfully imported llmtuner from src")
    except ImportError as e:
        print(f"Failed to import llmtuner: {e}")

# List contents of the src directory
print("\nContents of src directory:")
print(os.listdir(src_path))

Updated Python path:
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/LLaMA-Factory', '/content/LLaMA-Factory/src']
Successfully imported LLaMA Factory. Version: 0.9.1.dev0

Contents of src directory:
['train.py', 'api.py', 'webui.py', 'llamafactory', 'llamafactory.egg-info']


In [9]:
# LLaMA Factory Directory Structure Check

import os

llama_factory_path = '/content/LLaMA-Factory'

print("Contents of LLaMA-Factory directory:")
print(os.listdir(llama_factory_path))

print("\nContents of src directory:")
print(os.listdir(os.path.join(llama_factory_path, 'src')))

print("\nChecking for train.py file:")
train_py_path = os.path.join(llama_factory_path, 'src', 'train.py')
if os.path.exists(train_py_path):
    print(f"train.py found at {train_py_path}")
else:
    print("train.py not found in the expected location")

print("\nChecking llamafactory directory:")
llamafactory_path = os.path.join(llama_factory_path, 'src', 'llamafactory')
if os.path.exists(llamafactory_path):
    print("Contents of llamafactory directory:")
    print(os.listdir(llamafactory_path))
else:
    print("llamafactory directory not found")

Contents of LLaMA-Factory directory:
['.gitignore', 'docker', 'CITATION.cff', 'dpo_config.json', 'src', 'assets', 'Makefile', 'data', 'tests', '.env.local', 'LICENSE', 'requirements.txt', '.git', 'examples', '.github', 'scripts', 'MANIFEST.in', 'evaluation', 'pyproject.toml', 'setup.py', '.gitattributes', '.dockerignore', 'README_zh.md', 'README.md']

Contents of src directory:
['train.py', 'api.py', 'webui.py', 'llamafactory', 'llamafactory.egg-info']

Checking for train.py file:
train.py found at /content/LLaMA-Factory/src/train.py

Checking llamafactory directory:
Contents of llamafactory directory:
['launcher.py', 'data', 'eval', 'chat', 'hparams', 'cli.py', '__init__.py', '__pycache__', 'model', 'train', 'api', 'webui', 'extras']


In [17]:
# Final Corrected LLaMA Factory Colab DPO Script

import json
import os
from huggingface_hub import notebook_login

# Ensure we're in the correct directory
os.chdir('/content/LLaMA-Factory')

# Login to Hugging Face (if not already done)

# DPO configuration
dpo_args = {
    "stage": "dpo",
    "do_train": True,
    "model_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "adapter_name_or_path": "llama3_lora",
    "dataset": "anthropic_rlhf",
    "template": "llama3",
    "finetuning_type": "lora",
    "lora_target": "all",
    "output_dir": "llama3_dpo_lora",
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 10,
    "save_steps": 1000,
    "learning_rate": 5e-5,
    "num_train_epochs": 1.0,
    "max_samples": 500,
    "max_grad_norm": 1.0,
    "quantization_bit": 4,
    "fp16": True,
    "dpo_beta": 0.1
}

# Save configuration
config_path = "dpo_config.json"
json.dump(dpo_args, open(config_path, "w", encoding="utf-8"), indent=2)

# Construct the command with all necessary arguments
command = f"""
python src/train.py {config_path} \
    --stage dpo \
    --do_train \
    --model_name_or_path {dpo_args['model_name_or_path']} \
    --adapter_name_or_path {dpo_args['adapter_name_or_path']} \
    --dataset {dpo_args['dataset']} \
    --template {dpo_args['template']} \
    --finetuning_type {dpo_args['finetuning_type']} \
    --lora_target {dpo_args['lora_target']} \
    --output_dir {dpo_args['output_dir']} \
    --per_device_train_batch_size {dpo_args['per_device_train_batch_size']} \
    --gradient_accumulation_steps {dpo_args['gradient_accumulation_steps']} \
    --lr_scheduler_type {dpo_args['lr_scheduler_type']} \
    --logging_steps {dpo_args['logging_steps']} \
    --save_steps {dpo_args['save_steps']} \
    --learning_rate {dpo_args['learning_rate']} \
    --num_train_epochs {dpo_args['num_train_epochs']} \
    --max_samples {dpo_args['max_samples']} \
    --max_grad_norm {dpo_args['max_grad_norm']} \
    --quantization_bit {dpo_args['quantization_bit']} \
    --fp16 \
    --dpo_beta {dpo_args['dpo_beta']}
"""

# Run training
print("Starting DPO training...")
!{command}

print("DPO training completed.")

# Check if the output directory exists
if os.path.exists(dpo_args['output_dir']):
    print(f"Output directory '{dpo_args['output_dir']}' exists. Training was successful.")
    print("\nContents of output directory:")
    print(os.listdir(dpo_args['output_dir']))
else:
    print(f"Output directory '{dpo_args['output_dir']}' does not exist. Training might have failed.")

print("Script execution completed.")

Starting DPO training...
2024-09-30 19:36:59.643687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 19:36:59.664441: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 19:36:59.670383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 19:36:59.684508: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 19:37:00.

# 3. PPO Training

Proximal Policy Optimization (PPO) Training:
Similarly, configure the PPO training settings by defining the model path, dataset, and output directory.

In [18]:

import json
import os
from huggingface_hub import notebook_login

# Ensure we're in the correct directory
os.chdir('/content/LLaMA-Factory')

# Login to Hugging Face (if not already done)

# PPO configuration
ppo_args = {
    "stage": "ppo",
    "do_train": True,
    "model_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "adapter_name_or_path": "llama3_lora",  # Path to your SFT model
    "dataset": "anthropic_rlhf",
    "template": "llama3",
    "finetuning_type": "lora",
    "lora_target": "all",
    "output_dir": "llama3_ppo_lora",
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 10,
    "save_steps": 1000,
    "learning_rate": 1e-5,
    "num_train_epochs": 1.0,
    "max_samples": 500,
    "max_grad_norm": 1.0,
    "quantization_bit": 4,
    "fp16": True,
    "ppo_buffer_size": 1,
    "ppo_epochs": 4,
    "ppo_score_norm": True,
    "ppo_target": 6.0,
    "ppo_whiten_rewards": True,
    "ref_model": "unsloth/llama-3-8b-Instruct-bnb-4bit",  # Same as model_name_or_path for initial training
    "reward_model": "OpenAssistant/reward-model-deberta-v3-large-v2",
    "reward_model_type": "lora"
}

# Save configuration
config_path = "ppo_config.json"
json.dump(ppo_args, open(config_path, "w", encoding="utf-8"), indent=2)

print("Starting PPO training...")
!python src/train.py {config_path}

print("PPO training completed.")

# Check if the output directory exists
if os.path.exists(ppo_args['output_dir']):
    print(f"Output directory '{ppo_args['output_dir']}' exists. Training was successful.")
    print("\nContents of output directory:")
    print(os.listdir(ppo_args['output_dir']))
else:
    print(f"Output directory '{ppo_args['output_dir']}' does not exist. Training might have failed.")

print("Script execution completed.")

Starting PPO training...
2024-09-30 19:37:48.111798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 19:37:48.132458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 19:37:48.138664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 19:37:48.153498: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 19:37:49.